# Google Calendar API テスト

このノートブックでGoogle Calendar APIを使ってイベントを作成します。

## 事前準備
1. Google Cloud Consoleで OAuth 2.0 認証情報を作成
2. `credentials.json`をダウンロード
3. `composeApp/src/jvmMain/resources/credentials.json` に配置

**注意**: JVMツールと同じ`credentials.json`とトークンを共有します。

In [1]:
// 依存関係の追加
@file:DependsOn("com.google.api-client:google-api-client:2.0.0")
@file:DependsOn("com.google.oauth-client:google-oauth-client-jetty:1.39.0")
@file:DependsOn("com.google.apis:google-api-services-calendar:v3-rev20250404-2.0.0")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-datetime:0.6.2")









In [2]:
import com.google.api.client.auth.oauth2.Credential
import com.google.api.client.extensions.java6.auth.oauth2.AuthorizationCodeInstalledApp
import com.google.api.client.extensions.jetty.auth.oauth2.LocalServerReceiver
import com.google.api.client.googleapis.auth.oauth2.GoogleAuthorizationCodeFlow
import com.google.api.client.googleapis.auth.oauth2.GoogleClientSecrets
import com.google.api.client.googleapis.javanet.GoogleNetHttpTransport
import com.google.api.client.json.gson.GsonFactory
import com.google.api.client.util.store.FileDataStoreFactory
import com.google.api.services.calendar.Calendar
import com.google.api.services.calendar.CalendarScopes
import com.google.api.services.calendar.model.Event
import com.google.api.services.calendar.model.EventDateTime
import java.io.File
import java.io.FileReader
import kotlinx.datetime.*









In [3]:
// 設定
val applicationName = "Trip AI Calendar Test"
val jsonFactory = GsonFactory.getDefaultInstance()
// JVMツールと同じトークンディレクトリを使用
val tokensDirectoryPath = "${System.getProperty("user.home")}/.trip-ai/tokens"
val scopes = listOf(CalendarScopes.CALENDAR)
// JVMツールと同じcredentials.jsonを参照
val credentialsFilePath = "composeApp/src/jvmMain/resources/credentials.json"

In [4]:
/**
 * OAuth 2.0認証を実行してCredentialオブジェクトを取得
 */
fun getCredentials(): Credential {
    // credentials.jsonを読み込み
    val credentialsFile = File(credentialsFilePath)
    if (!credentialsFile.exists()) {
        error(
            """
            Error: credentials.json not found!
            
            セットアップ手順:
            1. Google Cloud Console (console.cloud.google.com) にアクセス
            2. 新しいプロジェクトを作成または既存のものを選択
            3. Google Calendar API を有効化
            4. OAuth 2.0 Client ID credentials を作成（デスクトップアプリケーション）
            5. credentials.json をダウンロード
            6. このノートブックと同じディレクトリに配置
        """.trimIndent()
        )
    }

    val clientSecrets = GoogleClientSecrets.load(jsonFactory, FileReader(credentialsFile))

    // 認証フローを構築
    val httpTransport = GoogleNetHttpTransport.newTrustedTransport()
    val flow = GoogleAuthorizationCodeFlow.Builder(
        httpTransport, jsonFactory, clientSecrets, scopes
    )
        .setDataStoreFactory(FileDataStoreFactory(File(tokensDirectoryPath)))
        .setAccessType("offline")
        .build()

    // ローカルサーバーでコールバックを受信（ポート8888）
    val receiver = LocalServerReceiver.Builder().setPort(8888).build()

    // ブラウザを開いて認証
    return AuthorizationCodeInstalledApp(flow, receiver).authorize("user")
}

println("認証関数を定義しました")

認証関数を定義しました


In [5]:
/**
 * Calendar APIクライアントを取得
 */
fun getCalendarService(): Calendar {
    val httpTransport = GoogleNetHttpTransport.newTrustedTransport()
    val credential = getCredentials()
    return Calendar.Builder(httpTransport, jsonFactory, credential)
        .setApplicationName(applicationName)
        .build()
}

println("Calendarサービス取得関数を定義しました")

Calendarサービス取得関数を定義しました


In [10]:
/**
 * カレンダーイベントを作成
 */
fun createEvent(eventName: String, dateTime: LocalDateTime): String {
    return try {
        val service = getCalendarService()

        // Kotlinの日時をGoogleの形式に変換
        val instant = dateTime.toInstant(TimeZone.currentSystemDefault())
        val eventDateTime = EventDateTime()
            .setDateTime(com.google.api.client.util.DateTime(instant.toEpochMilliseconds()))
            .setTimeZone(TimeZone.currentSystemDefault().id)

        // イベントオブジェクトを作成
        val event = Event()
            .setSummary(eventName)
            .setStart(eventDateTime)
            .setEnd(eventDateTime) // 同じ時刻で瞬間イベント

        // プライマリカレンダーにイベント挿入
        val calendarId = "primary"
        service.events().insert(calendarId, event).execute()

        "✅ カレンダーイベント作成成功!"
    } catch (e: Exception) {
        "❌ エラー: ${e.message}"
    }
}

println("イベント作成関数を定義しました")

イベント作成関数を定義しました


## テスト実行

以下のセルでイベントを作成します。初回実行時はブラウザが開き、Googleアカウントでの認証が必要です。

In [13]:
// テスト: 明日の午後2時にイベントを作成
val tomorrow = Clock.System.now()
    .plus(1, DateTimeUnit.DAY, TimeZone.currentSystemDefault())
    .toLocalDateTime(TimeZone.currentSystemDefault())
    .let { LocalDateTime(it.year, it.monthNumber, it.dayOfMonth, 14, 0) } // 14:00に設定

val result = createEvent("Trip AI テストイベント", tomorrow)
println(result)

✅ カレンダーイベント作成成功!


In [12]:
// カスタムイベントの作成例
// 日付と時刻を指定してイベントを作成できます

val customDateTime = LocalDateTime(
    year = 2025,
    monthNumber = 10,
    dayOfMonth = 20,
    hour = 15,
    minute = 30
)

val customResult = createEvent("札幌旅行: シマエナガ観察ツアー", customDateTime)
println(customResult)

✅ カレンダーイベント作成成功!


## トラブルシューティング

### credentials.json が見つからない場合
1. Google Cloud Console でプロジェクトを作成
2. Google Calendar API を有効化
3. OAuth 2.0 認証情報を作成（アプリケーションの種類: デスクトップアプリ）
4. credentials.json をダウンロード
5. `composeApp/src/jvmMain/resources/credentials.json` に配置

### 認証エラーの場合
- トークンをリセット: `~/.trip-ai/tokens/` ディレクトリを削除して再実行
- JVMツールと同じトークンを共有しているため、どちらか一方で認証すればOK

### ポート8888が使用中の場合
- `LocalServerReceiver.Builder().setPort(8888)` の8888を別のポート番号に変更